In [1]:
# Import necessary libraries
import pandas as pd
import random

# Load the dataset
data = pd.read_csv("synthetic_nourishnudge_data.csv")

# Print the dataset
print(data)


  Ingredient  Quantity    Unit    Category            Recipe Recipe Category  \
0    Chicken       113   grams    Proteins  Chicken Stir-Fry          Dinner   
1   Broccoli       100   grams  Vegetables    Broccoli Salad           Lunch   
2       Rice       199   grams      Grains       Tomato Soup            Soup   
3     Tomato        85  pieces      Fruits  Spinach Smoothie       Breakfast   
4    Spinach       141   grams  Vegetables        Rice Pilaf           Snack   

  Recipe Difficulty  Calories  Carbohydrates  Proteins  ...   Minerals  \
0              Easy       357              5        32  ...       Iron   
1            Medium       382             25        35  ...    Calcium   
2            Medium       355              8        39  ...  Potassium   
3              Easy       180             20        48  ...  Magnesium   
4              Hard       125             45        10  ...       Zinc   

  Dietary Preferences Dietary Restrictions  Health Goals Rating  \
0     I

In [2]:
# Define function to recommend ingredients based on user input
def recommend_ingredients(desired_dish, dietary_preferences=[], dietary_restrictions=[], health_goals=[]):
    # Filter dataset based on user's dietary preferences and restrictions
    filtered_data = data.copy()
    for preference in dietary_preferences:
        filtered_data = filtered_data[filtered_data['Dietary Preferences'].str.contains(preference)]
    for restriction in dietary_restrictions:
        filtered_data = filtered_data[~filtered_data['Dietary Restrictions'].str.contains(restriction)]
    
    # Filter by desired dish
    filtered_data = filtered_data[filtered_data['Recipe'].str.contains(desired_dish)]
    
    # If desired dish not found, recommend ingredients for a similar category
    if filtered_data.empty:
        filtered_data = data[data['Recipe Category'].str.contains(desired_dish)]
    
    print("Filtered Data:")
    print(filtered_data)
    
    # Recommend ingredients based on user's health goals
    if 'Weight Loss' in health_goals:
        # Filter by low-calorie ingredients
        filtered_data = filtered_data[filtered_data['Calories'] < 400]
    if 'Muscle Gain' in health_goals:
        # Filter by high-protein ingredients
        filtered_data = filtered_data[filtered_data['Proteins'] > 20]
    
    # Get recommended ingredients
    recommended_ingredients = filtered_data['Ingredient'].tolist()
    
    # Provide substitutions for common allergens or dietary preferences
    substitutions = {
        'Milk': ['Almond milk', 'Oat milk', 'Soy milk'],
        'Cheese': ['Vegan cheese', 'Nutritional yeast'],
        'Gluten-containing grains': ['Quinoa', 'Brown rice', 'Gluten-free pasta']
    }
    for ingredient, substitutes in substitutions.items():
        if ingredient in recommended_ingredients:
            recommended_ingredients.remove(ingredient)
            recommended_ingredients.extend(substitutes)
    
    return recommended_ingredients

# Example usage:
desired_dish = "Chicken Stir-Fry"
dietary_preferences = ["Italian Cuisine"]
dietary_restrictions = ["Dairy-Free"]
health_goals = ["Weight Loss"]

recommended_ingredients = recommend_ingredients(desired_dish, dietary_preferences, dietary_restrictions, health_goals)
print("Recommended Ingredients for", desired_dish, ":")
print(recommended_ingredients)

Filtered Data:
  Ingredient  Quantity   Unit  Category            Recipe Recipe Category  \
0    Chicken       113  grams  Proteins  Chicken Stir-Fry          Dinner   

  Recipe Difficulty  Calories  Carbohydrates  Proteins  ...  Minerals  \
0              Easy       357              5        32  ...      Iron   

  Dietary Preferences Dietary Restrictions Health Goals Rating    Comments  \
0     Italian Cuisine        Allergy: Nuts  Weight Loss      5  Delicious!   

          Ingredient Substitutions Serving Size Prep Time (mins)  \
0  Almond milk instead of cow milk            1               47   

   Cook Time (mins)  
0                96  

[1 rows x 22 columns]
Recommended Ingredients for Chicken Stir-Fry :
['Chicken']
